In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader

self.X = (X/255.).view(-1, 1, 96,96)

In [2]:
df = pd.read_csv('data/test.csv')

In [3]:
class FaceDataset(Dataset):
    def __init__(self):
        df = pd.read_csv('data/IdLookupTable.csv')
        X = []
        face_feature = {
    "left_eye_center_x": 1,
    "left_eye_center_y": 2,
    "right_eye_center_x": 3,
    "right_eye_center_y": 4,
    "left_eye_inner_corner_x": 5,
    "left_eye_inner_corner_y": 6,
    "left_eye_outer_corner_x": 7,
    "left_eye_outer_corner_y": 8,
    "right_eye_inner_corner_x": 9,
    "right_eye_inner_corner_y": 10,
    "right_eye_outer_corner_x": 11,
    "right_eye_outer_corner_y": 12,
    "left_eyebrow_inner_end_x": 13,
    "left_eyebrow_inner_end_y": 14,
    "left_eyebrow_outer_end_x": 15,
    "left_eyebrow_outer_end_y": 16,
    "right_eyebrow_inner_end_x": 17,
    "right_eyebrow_inner_end_y": 18,
    "right_eyebrow_outer_end_x": 19,
    "right_eyebrow_outer_end_y": 20,
    "nose_tip_x": 21,
    "nose_tip_y": 22,
    "mouth_left_corner_x": 23,
    "mouth_left_corner_y": 24,
    "mouth_right_corner_x": 25,
    "mouth_right_corner_y": 26,
    "mouth_center_top_lip_x": 27,
    "mouth_center_top_lip_y": 28,
    "mouth_center_bottom_lip_x": 29,
    "mouth_center_bottom_lip_y": 30
}

        for i,j in zip(df['ImageId'], df['FeatureName']):
            X.append((i,face_feature[j]))
        self.X = X
        # print(self.X)
        

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index]

test_dataset = FaceDataset()
val_dataloader = DataLoader(dataset=test_dataset, batch_size=1,num_workers=0)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 21 * 21, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 30)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 21 * 21)
        x = torch.relu(self.fc1(x))  
        x = torch.relu(self.fc2(x)) 
        x = self.fc3(x)
        return x   

In [5]:
model = torch.load('saves/model1.pt')

/tmp/ipykernel_122839/926347604.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('saves/model1.pt')


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [19]:
model.eval()
for i, (img_id, img_idx ) in enumerate(val_dataloader):
    print(i, img_id, img_idx)
    print(int(img_id))
    print(df['Image'][int(img_id)])
    y_pred = model(df['Image'][int(img_id)])
    

0 tensor([1]) tensor([1])
1
76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 47 37 33 32 33 35 50 55 54 50 51 61 78 92 100 101 79 55 47 52 50 47 39 38 52 46 25 25 39 52 63 59 67 79 68 48 44 44 43 44 45 53 56 55 50 48 54 58 51 52 67 72 86 76 64 75 97 97 83 85 81 67 53 45 39 33 33 53 73 60 44 47 72 98 121 120 117 127 120 115 117 68 85 84 72 63 52 57 69 69 49 34 40 46 57 63 58 45 39 30 31 35 47 52 47 52 53 54 70 84 93 104 94 67 50 48 49 48 39 35 48 53 44 33 35 47 66 73 73 73 69 58 48 42 41 51 52 54 58 51 49 53 60 59 49 46 61 72 87 82 66 71 93 108 95 83 85 81 64 58 54 42 33 48 66 57 45 53 72 88 110 125 125 134 132 111 106 63 78 84 76 64 52 53 58 56 47 37 41 48 57 64 66 56 44 38 28 32 41 49 42 46 56 56 64 75 83 91 97 82 60 49 50 47 42 38 43 56 60 52 41 48 62 73 72 63 61 64 56 44 44 45 50 50 53 59 63 64 62 60 51 51 60 74 87 85 60 65 86 104 105 88 86 86 74 64 65 56 49 58 68 66 55 57 74 88 98 116 127 139 133 110 106 65 73 80 75 65 59 54 53 48 41 36 39 50 58 69 72 61 49 46 41 37 39 42 42 42 51 58 6

TypeError: conv2d() received an invalid combination of arguments - got (str, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [8]:
model.forward([1])

TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [int]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [int]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
